In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient

In [2]:
qdrant_client = QdrantClient(host="localhost")
vectordb = Qdrant(
    client=qdrant_client, 
    collection_name="dandi_collection",
    embeddings=OpenAIEmbeddings(),
    content_payload_key="text_content",
)

In [4]:
vectordb.similarity_search("glial cells")

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/dandi_collection/points/search "HTTP/1.1 200 OK"


[Document(page_content='description: Whole-cell Patch-seq recordings from neurons of the mouse visual cortex from the Allen Institute for Brain Science, released in June 2020. The majority of cells in this dataset are GABAergic interneurons, but there are also a small number of glutamatergic neurons from layer 2/3 of the mouse visual cortex.', metadata={}),
 Document(page_content='title: Glia Accumulate Evidence that Actions Are Futile and Suppress Unsuccessful Behavior', metadata={}),
 Document(page_content='description: Data from "Physiological Properties and Behavioral Correlates of Hippocampal Granule Cells and Mossy Cells" Senzai, Buzsaki, Neuron 2017. Electrophysiology recordings of hippocampus during theta maze exploration.', metadata={}),
 Document(page_content='description: When a behavior repeatedly fails to achieve its goal, animals often give up and become passive, which can be strategic for preserving energy or regrouping between attempts. It is unknown how the brain ident

In [5]:
num_queries = 3
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(),
    llm=ChatOpenAI(temperature=0)
)

In [7]:
user_input = "I am interested in the tuning properties of glial cells. Are there any good dandisets for studying that?"
retriever_from_llm.generate_queries(user_input)

INFO:root:Generated queries: ['1. What are some recommended datasets for studying the tuning properties of glial cells?', '2. Can you suggest any datasets that are suitable for investigating the tuning properties of glial cells?', '3. Are there any specific datasets available that would be helpful in studying the tuning properties of glial cells?']


['1. What are some recommended datasets for studying the tuning properties of glial cells?',
 '2. Can you suggest any datasets that are suitable for investigating the tuning properties of glial cells?',
 '3. Are there any specific datasets available that would be helpful in studying the tuning properties of glial cells?']

In [8]:
unique_docs = retriever_from_llm.get_relevant_documents(question=user_input)
unique_docs

INFO:root:Generated queries: ['1. What are some recommended datasets for studying the tuning properties of glial cells?', '2. Can you suggest any datasets that are suitable for investigating the tuning properties of glial cells?', '3. Are there any specific datasets available that would be helpful in studying the tuning properties of glial cells?']
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/dandi_collection/points/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/dandi_collection/points/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/dandi_collection/points/search "HTTP/1.1 200 OK"


[Document(page_content='description: Whole-cell Patch-seq recordings from neurons of the mouse visual cortex from the Allen Institute for Brain Science, released in June 2020. The majority of cells in this dataset are GABAergic interneurons, but there are also a small number of glutamatergic neurons from layer 2/3 of the mouse visual cortex.', metadata={}),
 Document(page_content='description: Extracellular recordings from rabbit visual cortex neurons. Orientation testing with drifting grating stimuli. Hippocampal EEG for the state scoring procedure. ', metadata={}),
 Document(page_content='description: This dataset includes the electrophysiological (dorsal CA1 tetrodes) and behavioral (port triggers, reward delivery, and position tracking) data from Gillespie et al, Neuron 2021: "Hippocampal replay reflects specific past experiences rather than a plan for subsequent choice". For more information about this data, please contact Anna Gillespie or Loren Frank. ', metadata={}),
 Document(